In [10]:
import numpy as np
import pandas as pd

import ast
import random
import lmstudio as lms

from IPython.display import Markdown

In [11]:
raw_link = pd.read_csv('../../Data/postgenerate_gpt.csv')
raw_tthc = pd.read_csv('../../Data/raw_tthc.csv', dtype={'maThuTuc': str})

In [12]:
template_df = pd.read_csv('../../Template/template.csv', index_col=0)
config_df = pd.read_csv('../../Template/config.csv', index_col=0)

In [13]:
cols = ['link', 'Mã thủ tục hành chính liên quan', 'Số quyết định', 'Tên thủ tục', 'Cấp thực hiện', 'Loại thủ tục', 'Lĩnh vực', 
        'Trình tự thực hiện', 'Cách thức thực hiện', 'Thành phần hồ sơ', 'Đối tượng thực hiện', 'Cơ quan thực hiện', 
        'Cơ quan có thẩm quyền', 'Địa chỉ tiếp nhận hồ sơ', 'Cơ quan được ủy quyền', 'Cơ quan phối hợp', 'Kết quả thực hiện', 
        'Căn cứ pháp lý', 'Yêu cầu, điều kiện thực hiện', 'Từ khóa', 'Mô tả']

In [14]:
raw_tthc.columns = cols

In [15]:
pattern = []

pattern.append("Bạn đang cố gắng xác định xem câu trả lời có hiểu sai ngữ cảnh và mục đích của câu hỏi hay không.")
pattern.append("Bạn đang cố gắng xác định xem câu trả lời có sự mâu thuẫn về mặt thực tế so với kiến thức liên quan hay không. Một số thông tin trong câu trả lời có thể đã được bịa ra.")
pattern.append("Bạn đang cố gắng xác định xem câu trả lời có quá chung chung hoặc quá chi tiết so với mức độ cụ thể phù hợp để trả lời câu hỏi hay không.")
pattern.append("Bạn đang cố gắng xác định xem câu trả lời có thể được suy ra đúng từ kiến thức liên quan hay không.")

In [16]:
evaluate_context = template_df['evaluate_context']['open_source']
print(evaluate_context)

Bạn sẽ đóng vai trò là một người đánh giá câu trả lời (answer judge). Với một câu hỏi và câu trả lời, mục tiêu của bạn là xác định xem câu trả lời được cung cấp có chứa thông tin không đúng sự thật hoặc thông tin ảo giác (hallucinated information) hay không.
{pattern}
Bạn nên cố gắng hết sức để xác định xem câu trả lời có chứa thông tin không đúng sự thật hoặc thông tin ảo giác hay không. Câu trả lời bạn đưa ra bắt buộc CHỈ là "Có" hoặc "Không", và không giải thích gì thêm. Trả lời "Có" nếu câu trả lời chứa thông tin ảo giác, trả lời "Không" nếu câu trả lời không chứa thông tin ảo giác.



In [17]:
evaluate_template = template_df['evaluate_template']['open_source']
print(evaluate_template)

#Câu hỏi#: {question}

#Câu trả lời#: {answer}

#Đánh giá của bạn#:



In [ ]:
config = {"temperature": float(config_df['temperature']['open_source']), 
          "maxTokens": float(config_df['max_tokens']['open_source']), 
          "topPSampling": float(config_df['top_p']['open_source'])
          }
config

In [ ]:
model_name = "qwen2.5-7b-instruct-viet-sft"

model = lms.llm(model_name)

filename = f'{model_name}_evaluate.csv'

cols = ['link', 'cauTraLoiDung', 'danhGiaDung', 'cauTraLoiAoGiac', 'pattern', 'danhGiaAoGiac']

try:
    df = pd.read_csv(filename)
    if df.empty:
        print("CSV file is empty")
        df = pd.DataFrame(columns=cols)
except FileNotFoundError:
    print(f"File not found: {filename}")
    df = pd.DataFrame(columns=cols)
    df.to_csv(filename, index=False)
except pd.errors.EmptyDataError:
    print(f"File is completely empty: {filename}")
    df = pd.DataFrame(columns=cols)
    df.to_csv(filename, index=False)

for i in range(len(raw_link)):
    if raw_link['link'][i] in list(df['link'].values):
        continue

    p = raw_link['pattern'][i]
    question = raw_link['cauHoi'][i]
    right_answer = raw_link['cauTraLoi'][i]
    hallucinated_answer = raw_link['cauTraLoiAoGiac'][i]

    context_right = evaluate_context.format(pattern="")
    context_hallucinated = evaluate_context.format(pattern=pattern[p])

    prompt_right = evaluate_template.format(question=question, answer=hallucinated_answer)
    prompt_hallucinated = evaluate_template.format(question=question, answer=hallucinated_answer)

    try:
        completion = model.respond(history={"messages": [
            {"role": "system", "content": context_right},
            {"role": "user", "content": prompt_right}
        ]}, config=config)
        output_right = str(completion)
        completion = model.respond(history={"messages": [
            {"role": "system", "content": context_hallucinated},
            {"role": "user", "content": prompt_hallucinated}
        ]}, config=config)
        output_hallucinated = str(completion)
    except Exception as e:
        output_right = ""
        output_hallucinated = ""
        print(e)
        break

    results = [raw_link['link'][i], right_answer, output_right, hallucinated_answer, p, output_hallucinated]
    df.loc[i, cols] = results    
    df.to_csv(filename, index=False)

    print(f'Done of {len(raw_link)}: {i+1}', end='\r')

In [ ]:
df.info()
df.tail()